In [1]:
import keras
from keras import backend as K
import numpy as np
import io
import base64
from IPython.display import HTML
#import skvideo
#skvideo.setFFmpegPath("/usr/local/bin")
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$.

# Context

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [2]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

---
# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [38]:
class Environment(object):
    def __init__(self, grid_size=13, max_time=200, temperature=0.3, immob_penalty=0):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature 
        self.immob_penalty = immob_penalty
        # this will impact the number of positive and negative rewards in the environment

        # board on which one plays
        self.board = np.zeros((grid_size,grid_size)) # possible reward in each location on the board
        self.position = np.zeros((grid_size,grid_size)) # the very position of the rat at time t
        self.malus_position = np.zeros((self.grid_size, self.grid_size)) # recording malus rewards on visited locations
        
        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        

    def draw(self,e):
        skvideo.io.vwrite('runs/' + str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
        
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward += self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = - self.immob_penalty
        
        ### Then before giving the reward, we give the environment the possibility to change and act differently
        self.modify_board() # Everything needed can be found in self.x, self.y, self.board, self.t, etc...
        
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        bonus = 0.5*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2, :] = -1
        self.position[:, :2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0 # since we assume that we don't consider the first state in the reward,
        self.malus_position[self.x, self.y] = - self.immob_penalty # we can consider the original position as visited (or not ?)
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
    def modify_board():
        """This function modifies the board itself. It should use self.t and self.board"""
        pass

Let's define now the hyper parameters :

In [9]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open('runs/'+name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

Let's now define the training loop and the test loop :

In [42]:
def train(agent, env,epoch, eps_decay=None, eps_step=None, freq_video=10, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    name = prefix
    name = name + '_explore_' if eps_step and eps_decay else name
    
    eps = agent.epsilon

    for e in range(epoch):
        if eps_step and eps_decay and e!=0 and e%eps_step==0:
            eps = eps*eps_decay
            agent.set_epsilon(eps)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            # We modify this interval because we only care about the true reward 
            # and not the penalty of coming back to visited places. This penalty aims at helping the algorithm
            # itself but shouldn't be considered in the win/lose counts
            if reward > 0:
                win += reward
            if reward < -env.immob_penalty:
                lose -= reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if (e+1) % freq_video == 0:
            env.draw(name+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=name+'_model.h5',name_model=name+'_model.json')
        
        
#------------------------------------------------------------------------------------------------

def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1,epochs+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < -0.1:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/e))
    print('Final score: '+str(score/epochs))

## Random Agent

In [11]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(4)

Let's try it out ! *NB : This random agent cannot be trained so we test it directly*

In [13]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 10.0/17.0. Average score (-7.0)
Win/lose count 7.0/14.0. Average score (-7.0)
Win/lose count 7.5/16.0. Average score (-7.5)
Win/lose count 10.5/12.0. Average score (-6.0)
Win/lose count 8.0/11.0. Average score (-5.4)
Win/lose count 9.5/20.0. Average score (-6.25)
Win/lose count 11.5/13.0. Average score (-5.571428571428571)
Win/lose count 13.0/12.0. Average score (-4.75)
Win/lose count 12.0/15.0. Average score (-4.555555555555555)
Win/lose count 11.0/10.0. Average score (-4.0)
Final score: -4.0


> A random agent is not great and performs really badly... Let's head to a more suitable approach when actually learning something by reinforcement. However, this agent will be useful to draw comparisons when modifying the environment.

***
## DQN

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
We will need a class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. A maximum memory size is required to avoid side effects.

In [14]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>100:
            self.memory = self.memory[-100:]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

In [23]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=3):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.8 # previous : 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pred_rwd_per_action = self.model.predict(s.reshape(1,5,5,self.n_state))
        return np.argmax(pred_rwd_per_action)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            sample = self.memory.random_access()
            input_states[i,:,:,:] = sample[0]
            
            target_q[i, sample[2]] = sample[3]
            if not game_over_:
                target_q[i, sample[2]] += self.discount*np.max(self.model.predict(sample[1].reshape(1,5,5,self.n_state)).flatten())
        
        target_q = np.clip(target_q, -30, 30)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights('runs/'+name_weights, overwrite=True)
        with open('runs/'+name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open('runs/'+name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights('runs/'+name_weights)
        model.compile("sgd", "mse")
        self.model = model


def my_custom_loss(y_true,y_pred):
    """
    Let's have a look at the issue when using the MSE loss. Take for example the situation in which :
       - Target = [   0, 0.8,    0,   0]
       - Output = [-0.3, 0.1, -1.5, 4.1]
    Then the MSE loss will also consider the distance between 0 and the conrresponding output value even though
    0 means that we don't have the information since the action wasn't considered. Thus the algorithm is mislead
    with a wrong information and it actually learns either to reproduce it's output, or to only output values
    close to 0.
    
    What could be much more meaningful would be to only consider the squared deviation between the target and
    the output but only for the information that we have.
    """
    norm = K.sum(y_true, axis=-1)+1e-6 # Get the value of the non-zero element from the target
    norm = K.reshape(norm, shape=(-1,1)) # Reshape it in order to concatenate it after
    norm = K.concatenate([norm, norm, norm, norm],axis=-1) # Concatenate in order to allow the next operation
    loss = K.sum(K.square(y_true * (y_true - y_pred) / norm), axis=-1) # Multiply the deviation by the target
    # thus the non selected actions won't be considered in the loss (and divide by the norm of the target not 
    # to impact the final loss)
    return loss

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(32))
        model.add(Dense(64))
        model.add(Dense(4))
        
        loss = my_custom_loss #"mse"
        optimizer = Adam(lr=lr) #sgd(lr=lr, decay=1e-4, momentum=0.0), loss="mse")
        
        model.compile(optimizer=optimizer, loss=loss)
        self.model = model
        

In [24]:
K.clear_session()
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.0001, epsilon = 0.3, memory_size=2000, batch_size = 64)
train(agent, env, epoch=20, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

# we get impressively good results ! 
# It could be interesting to select the best set of parameters with a grid search

Epoch 001/020 | Loss 0.0179 | Win/lose count 5.5/11.0 (-5.5)
Epoch 002/020 | Loss 0.0799 | Win/lose count 3.5/7.0 (-3.5)
Epoch 003/020 | Loss 0.0307 | Win/lose count 7.0/12.0 (-5.0)
Epoch 004/020 | Loss 0.0007 | Win/lose count 14.0/13.0 (1.0)
Epoch 005/020 | Loss 0.0391 | Win/lose count 13.5/10.0 (3.5)
Epoch 006/020 | Loss 0.0000 | Win/lose count 7.5/3.0 (4.5)
Epoch 007/020 | Loss 0.0186 | Win/lose count 15.0/8.0 (7.0)
Epoch 008/020 | Loss 0.0233 | Win/lose count 7.5/3.0 (4.5)
Epoch 009/020 | Loss 0.0000 | Win/lose count 7.0/2.0 (5.0)
Epoch 010/020 | Loss 0.0177 | Win/lose count 12.0/6.0 (6.0)
Epoch 011/020 | Loss 0.0003 | Win/lose count 8.5/8.0 (0.5)
Epoch 012/020 | Loss 0.0042 | Win/lose count 8.0/5.0 (3.0)
Epoch 013/020 | Loss 0.0234 | Win/lose count 14.0/4.0 (10.0)
Epoch 014/020 | Loss 0.0348 | Win/lose count 12.0/6.0 (6.0)
Epoch 015/020 | Loss 0.0112 | Win/lose count 10.0/1.0 (9.0)
Epoch 016/020 | Loss 0.0120 | Win/lose count 8.5/8.0 (0.5)
Epoch 017/020 | Loss 0.0169 | Win/lose co

***
What if we were using a CNN instead ?

In [25]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(16))
        model.add(Dense(4))
        
        loss = my_custom_loss #"mse"
        optimizer = Adam(lr=lr) #sgd(lr=lr, decay=1e-4, momentum=0.0), loss="mse")
        
        model.compile(optimizer=optimizer, loss=loss)
        self.model = model

In [26]:
K.clear_session()
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.2, memory_size=2000, batch_size = 64)
train(agent, env, epoch=20, prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

### Comments :
# The loss represents the learning of the algorithm and the win/lose count represents somehow how well does the
# algorithm generalizes.

Epoch 001/020 | Loss 0.0091 | Win/lose count 9.0/10.0 (-1.0)
Epoch 002/020 | Loss 0.0931 | Win/lose count 13.0/9.0 (4.0)
Epoch 003/020 | Loss 0.0179 | Win/lose count 15.0/8.0 (7.0)
Epoch 004/020 | Loss 0.0050 | Win/lose count 19.0/5.0 (14.0)
Epoch 005/020 | Loss 0.2878 | Win/lose count 18.5/9.0 (9.5)
Epoch 006/020 | Loss 0.0989 | Win/lose count 16.0/3.0 (13.0)
Epoch 007/020 | Loss 0.1130 | Win/lose count 17.5/2.0 (15.5)
Epoch 008/020 | Loss 0.0000 | Win/lose count 9.5/8.0 (1.5)
Epoch 009/020 | Loss 0.0198 | Win/lose count 23.0/6.0 (17.0)
Epoch 010/020 | Loss 0.0455 | Win/lose count 16.5/4.0 (12.5)
Epoch 011/020 | Loss 0.2711 | Win/lose count 18.5/6.0 (12.5)
Epoch 012/020 | Loss 0.0063 | Win/lose count 18.0/4.0 (14.0)
Epoch 013/020 | Loss 0.0067 | Win/lose count 14.5/7.0 (7.5)
Epoch 014/020 | Loss 0.1494 | Win/lose count 24.5/5.0 (19.5)
Epoch 015/020 | Loss 0.0000 | Win/lose count 10.5/2.0 (8.5)
Epoch 016/020 | Loss 0.0002 | Win/lose count 19.0/3.0 (16.0)
Epoch 017/020 | Loss 0.0002 | W

***
***
Compare both algorithms' performances :

In [29]:
K.clear_session()
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_model.h5', name_model='cnn_train_model.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_train_model.h5', name_model='fc_train_model.json')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Win/lose count 11.0/5.0. Average score (6.0)
Win/lose count 15.0/4.0. Average score (8.5)
Win/lose count 14.5/5.0. Average score (8.833333333333334)
Win/lose count 15.5/2.0. Average score (10.0)
Win/lose count 14.5/0. Average score (10.9)
Win/lose count 19.5/2.0. Average score (12.0)
Win/lose count 15.5/1.0. Average score (12.357142857142858)
Win/lose count 18.5/6.0. Average score (12.375)
Win/lose count 17.0/3.0. Average score (12.555555555555555)
Win/lose count 21.0/6.0. Average score (12.8)
Final score: 12.8
Test of the FC
Win/lose count 6.5/4.0. Average score (2.5)
Win/lose count 7.5/3.0. Average score (3.5)
Win/lose count 12.5/2.0. Average score (5.833333333333333)
Win/lose count 7.5/3.0. Average score (5.5)
Win/lose count 15.0/1.0. Average score (7.2)
Win/lose count 5.5/0. Average score (6.916666666666667)
Win/lose count 6.5/0. Average score (6.857142857142857)
Win/lose count 17.0/3.0. Average score (7.75)
Win/lose count 13.5/3.0. Average score (8.055555555555555)

In [30]:
HTML(display_videos('cnn_test10.mp4'))

In [31]:
HTML(display_videos('fc_test10.mp4'))

> The local loss we introduced overcomes the issues related to the MSE one. In fact this is questionable since the MSE tends to make the network reproduce what it has done and penalizes bad moves (0 is better than negative rewards), but in our case, if we give a good exploration parameter we authorize the model to perform badly at first in order to get some cases to be able to predict future rewards. Once it has learned how to predict future rewards, it is thought to behave well but then would it get that bad rewards are worse than 0 ? This is questionable. In our case at least, it seems to work impressively well in very few epochs !!

***
The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. We use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not
---

In [32]:
# Training
K.clear_session()
env_eplore = Environment(grid_size=size, max_time=T, temperature=0.3, immob_penalty=0.1)
agent = DQN_CNN(size, lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, env, epoch=20, eps_decay=0.5, eps_step=3, prefix='cnn_train')
HTML(display_videos('cnn_train_explore_20.mp4'))

Epoch 001/020 | Loss 0.0120 | Win/lose count 10.5/10.0 (0.5)
Epoch 002/020 | Loss 0.0000 | Win/lose count 6.5/3.0 (3.5)
Epoch 003/020 | Loss 0.0021 | Win/lose count 13.0/10.0 (3.0)
Epoch 004/020 | Loss 0.0746 | Win/lose count 12.0/4.0 (8.0)
Epoch 005/020 | Loss 0.0022 | Win/lose count 10.0/1.0 (9.0)
Epoch 006/020 | Loss 0.0090 | Win/lose count 11.5/7.0 (4.5)
Epoch 007/020 | Loss 0.0663 | Win/lose count 18.5/2.0 (16.5)
Epoch 008/020 | Loss 0.0000 | Win/lose count 14.0/3.0 (11.0)
Epoch 009/020 | Loss 0.1558 | Win/lose count 8.5/4.0 (4.5)
Epoch 010/020 | Loss 0.1008 | Win/lose count 13.5/2.0 (11.5)
Epoch 011/020 | Loss 0.1920 | Win/lose count 16.5/3.0 (13.5)
Epoch 012/020 | Loss 0.1058 | Win/lose count 15.5/4.0 (11.5)
Epoch 013/020 | Loss 0.0020 | Win/lose count 18.5/0 (18.5)
Epoch 014/020 | Loss 0.1021 | Win/lose count 17.0/2.0 (15.0)
Epoch 015/020 | Loss 0.0276 | Win/lose count 13.5/0 (13.5)
Epoch 016/020 | Loss 0.2595 | Win/lose count 22.5/0 (22.5)
Epoch 017/020 | Loss 0.0306 | Win/los

In [33]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 8.5/0. Average score (8.5)
Win/lose count 23.5/2.0. Average score (15.0)
Win/lose count 24.0/2.0. Average score (17.333333333333332)
Win/lose count 4.0/2.0. Average score (13.5)
Win/lose count 8.5/2.0. Average score (12.1)
Win/lose count 9.0/0. Average score (11.583333333333334)
Win/lose count 8.0/1.0. Average score (10.928571428571429)
Win/lose count 12.5/1.0. Average score (11.0)
Win/lose count 0.5/0. Average score (9.833333333333334)
Win/lose count 17.0/1.0. Average score (10.45)
Final score: 10.45


> This approach works impressively well ! Indeed there is a local minimum in our original approach which needed to be avoided : since the model itself tries to be confident about its prediction somehow, one can think of the situation in which the mouse goes back and forth on 2 possible positions. Indeed, it will predict that the expected future reward is 0, which is true since it doesn't explore anymore and thus with a current reward of 0 and a future reward of 0, the model is able to predict very precisely its future and thus is in a local minimum. We need to avoid this situation by forcing an exploration. We need to put a high exploration parameter at first and then decrease it when the model has enough data to train on. The other part of the new approach deals with giving a minus reward when coming back on visited locations, which is great since the model is not able to predict the future reward precisely (infinite instead of 0).

---
---
---
# Adaptive Environment

To be tested :
- Modify the environment itself :
    - Translation of the board every T time step by a factor to be set as parameter
    - Based on what the rat eats, his vision can increase in dimension
    - New rewards pop on the board based on the rat's actions
    - Random flip of the board based on a central/axial symmetry every T time step.
    - New random board at every time step. (should break the algorithm and actually be pretty interesting considering the fact that the DQN approach with a CNN tries to be confident over its predictions... How can it be in this situation ?)
- Rewards follow a distribution of probability rather than being fixed : cheese ~ N(0.5, 1) | poison ~ N(-1, 0.5) for instance.
- Adverse environnement (environnement acts against the player). Implement and explore the Exp3 algorithm and compare it to the UCB1 one.

In [39]:
class Adaptive_Environment(Environment):
    def __init__(self, *args, grid_size=13, max_time=200, temperature=0.3, transform_board=None, **kwargs):
        super(Adaptive_Environment, self).__init__(*args,grid_size=grid_size, max_time=max_time, 
                                                   temperature=temperature,**kwargs)
        
        self.transform_board = transform_board

    def modify_board(self):
        """This function modifies the board itself. It should use self.t and self.board.
        To do so, we consider the transform_board function passed to the adaptative environment.
        Such a function should take as input the arguments of `self` and return the new board.
        """
        if self.transform_board is not None:
            self.board = self.transform_board(infos=self)
        else:
            pass

In [40]:
def board_translation(infos, time_step=1, amplitude=1):
    """Translate the board
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        time_step : int
            Frequency at which we want to modify the board.
        amplitude : int
            Number of columns (or rows, it doesn't matter given the symmetry of the game) to consider
            to shift the board.
            
    output :
    ------
        a new version of the board contained in the object self
    """
    board = infos.board
    if infos.t%time_step==0: # done after self.t += 1 so the case t=0 doesn't need to be considered
        # for instance with an amplitude of 2 : board[:, 2:] = board[:, :-2] | board[:, :2] = board[:, -2:]
        board[:, amplitude:], board[:, :amplitude] = board[:, :-amplitude], board[:, -amplitude:]
    return board

In [43]:
K.clear_session()
translated_env = Adaptive_Environment(grid_size=size, max_time=T, temperature=0.3, immob_penalty=0.1,
                                      transform_board=board_translation)
agent = DQN_CNN(size, lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, translated_env, epoch=1, eps_decay=0.5, eps_step=3, freq_video=1, prefix='translate_env')
HTML(display_videos('translate_env_explore_1.mp4'))

Epoch 001/001 | Loss 0.0106 | Win/lose count 4.1/14.899999999999999 (-10.799999999999999)
